### Библиотеки

In [23]:
import numpy as np
import random

### Реализации алгоритмов Нидлмана-Вунша и Смита-Вотермана

In [55]:
def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, gap=-1):
    n, m = len(seq1), len(seq2)
    score_matrix = np.zeros((n+1, m+1))
    traceback = np.zeros((n+1, m+1), dtype=object)
    
    for i in range(n+1):
        score_matrix[i][0] = i * gap
        traceback[i][0] = "U"
    for j in range(m+1):
        score_matrix[0][j] = j * gap
        traceback[0][j] = "L"
    
    traceback[0][0] = "END"
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            match_score = score_matrix[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            delete = score_matrix[i-1][j] + gap
            insert = score_matrix[i][j-1] + gap
            max_score = max(match_score, delete, insert)
            score_matrix[i][j] = max_score
            
            if max_score == match_score:
                traceback[i][j] = "D"
            elif max_score == delete:
                traceback[i][j] = "U"
            else:
                traceback[i][j] = "L"
    
    return score_matrix, traceback

In [56]:
def traceback_needleman_wunsch(seq1, seq2, traceback):
    aligned_seq1, aligned_seq2 = "", ""
    i, j = len(seq1), len(seq2)
    while traceback[i][j] != "END":
        if traceback[i][j] == "D":
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            i -= 1
            j -= 1
        elif traceback[i][j] == "U":
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            j -= 1
    return aligned_seq1, aligned_seq2

In [57]:
def smith_waterman(seq1, seq2, match=1, mismatch=-1, gap=-1):
    n, m = len(seq1), len(seq2)
    score_matrix = np.zeros((n+1, m+1))
    max_i, max_j, max_score = 0, 0, 0
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            match_score = score_matrix[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            delete = score_matrix[i-1][j] + gap
            insert = score_matrix[i][j-1] + gap
            score_matrix[i][j] = max(0, match_score, delete, insert)
            
            if score_matrix[i][j] > max_score:
                max_score = score_matrix[i][j]
                max_i, max_j = i, j
    
    return score_matrix, max_i, max_j

In [58]:
def traceback_smith_waterman(seq1, seq2, score_matrix, max_i, max_j, match=1, mismatch=-1, gap=-1):
    aligned_seq1, aligned_seq2 = "", ""
    i, j = max_i, max_j
    while i > 0 and j > 0 and score_matrix[i][j] > 0:
        match_score = score_matrix[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
        if score_matrix[i][j] == match_score:
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            i -= 1
            j -= 1
        elif score_matrix[i][j] == score_matrix[i-1][j] + gap:
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            j -= 1
    
    return aligned_seq1, aligned_seq2

### Примеры

In [59]:
seq1 = "GACGAAG"
seq2 = "ACCAAG"
print("Последовательности:" + seq1 + ", " + seq2)

print("\nМатрица Нидлмана-Вунша:")
nw_matrix, nw_traceback = needleman_wunsch(seq1, seq2)
print(nw_matrix)
nw_aln1, nw_aln2 = traceback_needleman_wunsch(seq1, seq2, nw_traceback)
print("Глобальное выравнивание:")
print(nw_aln1)
print(nw_aln2)

print("\nМатрица Смита-Вотермана:")
sw_matrix, max_i, max_j = smith_waterman(seq1, seq2)
print(sw_matrix)
sw_aln1, sw_aln2 = traceback_smith_waterman(seq1, seq2, sw_matrix, max_i, max_j)
print("Локальное выравнивание:")
print(sw_aln1)
print(sw_aln2)

Последовательности:GACGAAG, ACCAAG

Матрица Нидлмана-Вунша:
[[ 0. -1. -2. -3. -4. -5. -6.]
 [-1. -1. -2. -3. -4. -5. -4.]
 [-2.  0. -1. -2. -2. -3. -4.]
 [-3. -1.  1.  0. -1. -2. -3.]
 [-4. -2.  0.  0. -1. -2. -1.]
 [-5. -3. -1. -1.  1.  0. -1.]
 [-6. -4. -2. -2.  0.  2.  1.]
 [-7. -5. -3. -3. -1.  1.  3.]]
Глобальное выравнивание:
GACGAAG
-ACCAAG

Матрица Смита-Вотермана:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 1. 0.]
 [0. 0. 2. 1. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 1.]
 [0. 1. 0. 0. 2. 1. 0.]
 [0. 1. 0. 0. 1. 3. 2.]
 [0. 0. 0. 0. 0. 2. 4.]]
Локальное выравнивание:
ACGAAG
ACCAAG


In [60]:
seq1 = "GAACGTAGCGC"
seq2 = "GAACGAAGCCCC"
print("Последовательности:" + seq1 + ", " + seq2)

print("\nМатрица Нидлмана-Вунша:")
nw_matrix, nw_traceback = needleman_wunsch(seq1, seq2)
print(nw_matrix)
nw_aln1, nw_aln2 = traceback_needleman_wunsch(seq1, seq2, nw_traceback)
print("Глобальное выравнивание:")
print(nw_aln1)
print(nw_aln2)

print("\nМатрица Смита-Вотермана:")
sw_matrix, max_i, max_j = smith_waterman(seq1, seq2)
print(sw_matrix)
sw_aln1, sw_aln2 = traceback_smith_waterman(seq1, seq2, sw_matrix, max_i, max_j)
print("Локальное выравнивание:")
print(sw_aln1)
print(sw_aln2)

Последовательности:GAACGTAGCGC, GAACGAAGCCCC

Матрица Нидлмана-Вунша:
[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12.]
 [ -1.   1.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10.]
 [ -2.   0.   2.   1.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.]
 [ -3.  -1.   1.   3.   2.   1.   0.  -1.  -2.  -3.  -4.  -5.  -6.]
 [ -4.  -2.   0.   2.   4.   3.   2.   1.   0.  -1.  -2.  -3.  -4.]
 [ -5.  -3.  -1.   1.   3.   5.   4.   3.   2.   1.   0.  -1.  -2.]
 [ -6.  -4.  -2.   0.   2.   4.   4.   3.   2.   1.   0.  -1.  -2.]
 [ -7.  -5.  -3.  -1.   1.   3.   5.   5.   4.   3.   2.   1.   0.]
 [ -8.  -6.  -4.  -2.   0.   2.   4.   4.   6.   5.   4.   3.   2.]
 [ -9.  -7.  -5.  -3.  -1.   1.   3.   3.   5.   7.   6.   5.   4.]
 [-10.  -8.  -6.  -4.  -2.   0.   2.   2.   4.   6.   6.   5.   4.]
 [-11.  -9.  -7.  -5.  -3.  -1.   1.   1.   3.   5.   7.   7.   6.]]
Глобальное выравнивание:
GAACGTAG-CGC
GAACGAAGCCCC

Матрица Смита-Вотермана:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [61]:
seq1 = "ACGGCTGACGCGAT"
seq2 = "GGGCAGCAGTCAG"
print("Последовательности:" + seq1 + ", " + seq2)

print("\nМатрица Нидлмана-Вунша:")
nw_matrix, nw_traceback = needleman_wunsch(seq1, seq2)
print(nw_matrix)
nw_aln1, nw_aln2 = traceback_needleman_wunsch(seq1, seq2, nw_traceback)
print("Глобальное выравнивание:")
print(nw_aln1)
print(nw_aln2)

print("\nМатрица Смита-Вотермана:")
sw_matrix, max_i, max_j = smith_waterman(seq1, seq2)
print(sw_matrix)
sw_aln1, sw_aln2 = traceback_smith_waterman(seq1, seq2, sw_matrix, max_i, max_j)
print("Локальное выравнивание:")
print(sw_aln1)
print(sw_aln2)

Последовательности:ACGGCTGACGCGAT, GGGCAGCAGTCAG

Матрица Нидлмана-Вунша:
[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13.]
 [ -1.  -1.  -2.  -3.  -4.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11.]
 [ -2.  -2.  -2.  -3.  -2.  -3.  -4.  -3.  -4.  -5.  -6.  -7.  -8.  -9.]
 [ -3.  -1.  -1.  -1.  -2.  -3.  -2.  -3.  -4.  -3.  -4.  -5.  -6.  -7.]
 [ -4.  -2.   0.   0.  -1.  -2.  -2.  -3.  -4.  -3.  -4.  -5.  -6.  -5.]
 [ -5.  -3.  -1.  -1.   1.   0.  -1.  -1.  -2.  -3.  -4.  -3.  -4.  -5.]
 [ -6.  -4.  -2.  -2.   0.   0.  -1.  -2.  -2.  -3.  -2.  -3.  -4.  -5.]
 [ -7.  -5.  -3.  -1.  -1.  -1.   1.   0.  -1.  -1.  -2.  -3.  -4.  -3.]
 [ -8.  -6.  -4.  -2.  -2.   0.   0.   0.   1.   0.  -1.  -2.  -2.  -3.]
 [ -9.  -7.  -5.  -3.  -1.  -1.  -1.   1.   0.   0.  -1.   0.  -1.  -2.]
 [-10.  -8.  -6.  -4.  -2.  -2.   0.   0.   0.   1.   0.  -1.  -1.   0.]
 [-11.  -9.  -7.  -5.  -3.  -3.  -1.   1.   0.   0.   0.   1.   0.  -1.]
 [-12. -10.  -8.  -6.  -4.  -4.  -2.   0.   0.   1

### Выравнивание через Python

In [37]:
from Bio.Align import Alignment
from Bio import Align

In [48]:
seq1 = "GACGAAG"
seq2 = "ACCAAG"

In [62]:
seq3 = "GAACGTAGCGC"
seq4 = "GAACGAAGCCCC"

In [63]:
seq5 = "ACGGCTGACGCGAT"
seq6 = "GGGCAGCAGTCAG"

#### Смит-Вотерман

In [49]:
aligner_1 = Align.PairwiseAligner()

aligner_1.mode = "local"  # Локальное выравнивание
aligner_1.match_score = 1.0  
aligner_1.mismatch_score = -1.0 
aligner_1.open_gap_score = -1.0 
aligner_1.extend_gap_score = -1.0 

In [50]:
alignments_1 = aligner_1.align(seq1, seq2)

In [51]:
for alignment in alignments_1:

    print(alignment)

target            1 ACGAAG 7
                  0 ||.||| 6
query             0 ACCAAG 6



In [64]:
alignments_1 = aligner_1.align(seq3, seq4)

In [65]:
for alignment in alignments_1:

    print(alignment)

target            0 GAACGTAGC 9
                  0 |||||.||| 9
query             0 GAACGAAGC 9



In [66]:
alignments_1 = aligner_1.align(seq5, seq6)

In [67]:
for alignment in alignments_1:

    print(alignment)

target            2 GGC 5
                  0 ||| 3
query             1 GGC 4



#### Нидлман-Вунш

In [52]:
aligner_2 = Align.PairwiseAligner()

aligner_2.mode = "global"  # Глобальное выравнивание
aligner_2.match_score = 1.0
aligner_2.mismatch_score = -1.0
aligner_2.open_gap_score = -1.0
aligner_2.extend_gap_score = -1.0

In [53]:
alignments_2 = aligner_2.align(seq1, seq2)

In [54]:
for alignment in alignments_2:

    print(alignment)

target            0 GACGAAG 7
                  0 -||.||| 7
query             0 -ACCAAG 6



In [68]:
alignments_2 = aligner_2.align(seq3, seq4)

In [69]:
for alignment in alignments_2:

    print(alignment)

target            0 GAACGTAGCGC- 11
                  0 |||||.|||.|- 12
query             0 GAACGAAGCCCC 12

target            0 GAACGTAGCG-C 11
                  0 |||||.|||.-| 12
query             0 GAACGAAGCCCC 12

target            0 GAACGTAGC-GC 11
                  0 |||||.|||-.| 12
query             0 GAACGAAGCCCC 12

target            0 GAACGTAG-CGC 11
                  0 |||||.||-|.| 12
query             0 GAACGAAGCCCC 12



In [70]:
alignments_2 = aligner_2.align(seq5, seq6)

In [71]:
for alignment in alignments_2:

    print(alignment)

target            0 ACGGCTGAC-G-CGAT 14
                  0 .-|||.|-|-|-|-|. 16
query             0 G-GGCAG-CAGTC-AG 13

target            0 ACGGCTGAC-G-CGAT 14
                  0 -.|||.|-|-|-|-|. 16
query             0 -GGGCAG-CAGTC-AG 13

target            0 ACGGCTG-ACG-CGAT 14
                  0 .-|||.|-|-|-|-|. 16
query             0 G-GGCAGCA-GTC-AG 13

target            0 ACGGCTG-ACG-CGAT 14
                  0 -.|||.|-|-|-|-|. 16
query             0 -GGGCAGCA-GTC-AG 13

